In [1]:
import glob
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp
import os
from build import build_model

In [2]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [3]:
# !bash report.sh
organisms = set(pandas.read_csv("biomass_constrained.txt",index_col=0,header=None).index.to_list())
done = {i.split(".csv")[0] for i in os.listdir("./cases/overflow/high_FA/")}
zinc_in_polymerase = set(pandas.read_csv("zinc_in_polymerase.txt",index_col=0,header=None).index.to_list())

In [4]:
run_for = organisms
len(run_for)

495

In [7]:
exchanges = ["EX_ac(e)","EX_but(e)","EX_ppa(e)"]

In [20]:
from diets import load_model,constrain_exchanges,get_bounds
from overflow import get_overflow
import coralme

conditions = ["high"]
metabolite = "FA"

for c in conditions:
    name = "{}_{}".format(c,metabolite)
    fluxdir = "./cases/fluxes/{}".format(name)
    overflowdir = "./cases/overflow/{}".format(name)
    if not os.path.isdir(fluxdir):
        os.makedirs(fluxdir)
        os.makedirs(overflowdir)

def run(org):
    model = coralme.io.pickle.load_pickle_me_model("./me-models/{}/MEModel-BIO-{}-ME-TS.pkl".format(org,org))
    return model
    sol0 = model.solution
    df0 = sol0.to_frame()
    # Exchanges
    rlist = exchanges
    # Run for all conditions
    for condition in conditions:
        name = "{}_{}".format(condition,metabolite)
        dct = {k:-2.0 for k in rlist if k in model.reactions}
        constrain_exchanges(model,dct)
        model.optimize()
        model.solution.to_frame().to_csv("cases/fluxes/{}/{}.csv".format(name,org))
        get_overflow(model).to_csv("./cases/overflow/{}/{}.csv".format(name,org))

In [ ]:
NP = min([10,len(run_for)])
pool = mp.Pool(NP,maxtasksperchild=1)
pbar = tqdm(total=len(run_for),position=0,leave=True)
pbar.set_description('Building ({} threads)'.format(NP))
def collect_result(result):
    pbar.update(1)

for org in run_for:
    args = ([org])
    pool.apply_async(run,args, callback=collect_result)
pool.close()
pool.join()

Building (2 threads):   0%|                               | 0/2 [00:00<?, ?it/s]

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmpl0o531wt.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Read LP format model from file /tmp/tmp3gwjt_5d.lp
Reading time = 0.00 seconds
: 832 rows, 1788 columns, 7876 nonzeros
Read LP format model from file /tmp/tmpr0uy6cf7.lp
Reading time = 0.00 seconds
: 854 rows, 1786 columns, 7706 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmpde7k91in.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Iteration	 Solution to check	Solver Status
---------	------------------	-------------
Read LP format model from file /tmp/tmp232qcy36.lp
Reading time = 0.00 seconds
: 1301 rows, 3132 columns, 13778 nonzeros
        1	1.4050280687025918	Not feasible
Read LP format model from file /tmp/tmpntqwavw1.lp
Reading time = 0.00 seconds
: 1316 rows, 3130 columns, 136

Building (2 threads):  50%|███████████           | 1/2 [01:56<01:56, 116.73s/it]

        2	0.7025140343512959	Not feasible
        3	0.3512570171756479	Not feasible
        4	0.1756285085878240	Not feasible
        5	0.0878142542939120	Not feasible
        6	0.0439071271469560	Not feasible
        7	0.0219535635734780	Not feasible
        8	0.0109767817867390	Not feasible
        9	0.0054883908933695	Not feasible
       10	0.0027441954466847	Not feasible
       11	0.0013720977233424	Optimal
       12	0.0020581465850136	Optimal
       13	0.0024011710158492	Optimal
       14	0.0025726832312670	Optimal
